In [8]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler
from category_encoders.target_encoder import TargetEncoder

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error,mean_squared_log_error,make_scorer
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor, Pool, cv

In [18]:
train = pd.read_csv("train2.csv").drop(['사망자수','중상자수','경상자수','부상자수'],axis=1)
test = pd.read_csv("test(통합본).csv")

In [19]:
def gu(address):
    return address.split()[0]
def dong(address):
    return address.split()[1]

In [20]:
for df in [train,test]:
    df['구'] = df['주소'].apply(gu)
    df['동'] = df['주소'].apply(dong)
    df.drop('주소',axis=1,inplace=True)
    

In [21]:
train.columns

Index(['요일', '기상상태', '도로형태', '노면상태', '사고유형', '연휴', '년도', '월', '일', '시간',
       'ECLO', '설치장소', '설치개수', 'CCTV설치대수', '급지구분_1', '급지구분_2', '급지구분_3', '구',
       '동'],
      dtype='object')

In [22]:
train = train[['요일', '기상상태', '도로형태', '노면상태', '사고유형', '구', '동', '연휴', '년도', '월', '일', '시간',
       '설치장소', '설치개수', 'CCTV설치대수', '급지구분_1', '급지구분_2', '급지구분_3', 'ECLO']]
test = test[['요일', '기상상태', '도로형태', '노면상태', '사고유형', '구', '동', '연휴', '년도', '월', '일', '시간',
       '설치장소', '설치개수', 'CCTV설치대수', '급지구분_1', '급지구분_2', '급지구분_3']]

In [23]:
categorical_features = list(train.dtypes[train.dtypes == "object"].index)
display(categorical_features)

for i in categorical_features:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for case in np.unique(test[i]):
        if case not in le.classes_: 
            le.classes_ = np.append(le.classes_, case) 
    test[i]=le.transform(test[i])

['요일', '기상상태', '도로형태', '노면상태', '사고유형', '구', '동']

In [24]:
X = train.drop('ECLO',axis=1).values
y = train['ECLO'].values
# y = np.log1p(y)

In [25]:
st = StandardScaler()
X_standard = st.fit_transform(X)
test_standard = st.transform(test.iloc[:,:])
x_train,x_test,y_train,y_test = train_test_split(X_standard,y,test_size=0.2)

In [26]:
def rmsle(predicted_values, actual_values):
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)

    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)

    difference = log_predict - log_actual
    difference = np.square(difference)
    mean_difference = difference.mean()
    score = np.sqrt(mean_difference)

    return score

rmsle_scorer = make_scorer(rmsle)

In [109]:
cat = CatBoostRegressor()

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9],
       }

grid_search_result = cat.grid_search(grid,x_train,y_train,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.6505815	test: 1.6465050	best: 1.6465050 (0)	total: 2.87ms	remaining: 2.87s
1:	learn: 1.6047865	test: 1.6007983	best: 1.6007983 (1)	total: 5.91ms	remaining: 2.95s
2:	learn: 1.5605287	test: 1.5565828	best: 1.5565828 (2)	total: 8.04ms	remaining: 2.67s
3:	learn: 1.5176586	test: 1.5137831	best: 1.5137831 (3)	total: 10.2ms	remaining: 2.53s
4:	learn: 1.4762214	test: 1.4724170	best: 1.4724170 (4)	total: 14.6ms	remaining: 2.91s
5:	learn: 1.4361570	test: 1.4324105	best: 1.4324105 (5)	total: 17ms	remaining: 2.82s
6:	learn: 1.3974035	test: 1.3937109	best: 1.3937109 (6)	total: 19.1ms	remaining: 2.71s
7:	learn: 1.3599239	test: 1.3563114	best: 1.3563114 (7)	total: 21.2ms	remaining: 2.62s
8:	learn: 1.3236974	test: 1.3201387	best: 1.3201387 (8)	total: 23.3ms	remaining: 2.57s
9:	learn: 1.2886845	test: 1.2851869	best: 1.2851869 (9)	total: 25.3ms	remaining: 2.51s
10:	learn: 1.2547772	test: 1.2513583	best: 1.2513583 (10)	total: 27.5ms	remaining: 2.47s
11:	learn: 1.2220716	test: 1.2187383	best: 

188:	learn: 0.4438485	test: 0.4506311	best: 0.4506311 (188)	total: 404ms	remaining: 1.73s
189:	learn: 0.4438312	test: 0.4506282	best: 0.4506282 (189)	total: 406ms	remaining: 1.73s
190:	learn: 0.4438099	test: 0.4506356	best: 0.4506282 (189)	total: 408ms	remaining: 1.73s
191:	learn: 0.4437895	test: 0.4506264	best: 0.4506264 (191)	total: 410ms	remaining: 1.73s
192:	learn: 0.4437669	test: 0.4506289	best: 0.4506264 (191)	total: 412ms	remaining: 1.72s
193:	learn: 0.4437475	test: 0.4506402	best: 0.4506264 (191)	total: 414ms	remaining: 1.72s
194:	learn: 0.4437226	test: 0.4506234	best: 0.4506234 (194)	total: 416ms	remaining: 1.72s
195:	learn: 0.4437065	test: 0.4506178	best: 0.4506178 (195)	total: 418ms	remaining: 1.71s
196:	learn: 0.4436859	test: 0.4506066	best: 0.4506066 (196)	total: 420ms	remaining: 1.71s
197:	learn: 0.4436599	test: 0.4506151	best: 0.4506066 (196)	total: 422ms	remaining: 1.71s
198:	learn: 0.4436373	test: 0.4506047	best: 0.4506047 (198)	total: 424ms	remaining: 1.71s
199:	learn

292:	learn: 0.4420354	test: 0.4503034	best: 0.4502876 (289)	total: 617ms	remaining: 1.49s
293:	learn: 0.4420268	test: 0.4503007	best: 0.4502876 (289)	total: 619ms	remaining: 1.49s
294:	learn: 0.4420187	test: 0.4502969	best: 0.4502876 (289)	total: 621ms	remaining: 1.48s
295:	learn: 0.4420012	test: 0.4502976	best: 0.4502876 (289)	total: 623ms	remaining: 1.48s
296:	learn: 0.4419862	test: 0.4502928	best: 0.4502876 (289)	total: 625ms	remaining: 1.48s
297:	learn: 0.4419763	test: 0.4502864	best: 0.4502864 (297)	total: 627ms	remaining: 1.48s
298:	learn: 0.4419612	test: 0.4502910	best: 0.4502864 (297)	total: 629ms	remaining: 1.47s
299:	learn: 0.4419504	test: 0.4502911	best: 0.4502864 (297)	total: 631ms	remaining: 1.47s
300:	learn: 0.4419346	test: 0.4502929	best: 0.4502864 (297)	total: 633ms	remaining: 1.47s
301:	learn: 0.4419166	test: 0.4502859	best: 0.4502859 (301)	total: 635ms	remaining: 1.47s
302:	learn: 0.4419039	test: 0.4502798	best: 0.4502798 (302)	total: 637ms	remaining: 1.47s
303:	learn

388:	learn: 0.4405416	test: 0.4501117	best: 0.4500960 (385)	total: 812ms	remaining: 1.27s
389:	learn: 0.4405218	test: 0.4501152	best: 0.4500960 (385)	total: 814ms	remaining: 1.27s
390:	learn: 0.4405052	test: 0.4501065	best: 0.4500960 (385)	total: 817ms	remaining: 1.27s
391:	learn: 0.4404886	test: 0.4500939	best: 0.4500939 (391)	total: 818ms	remaining: 1.27s
392:	learn: 0.4404742	test: 0.4500865	best: 0.4500865 (392)	total: 821ms	remaining: 1.27s
393:	learn: 0.4404553	test: 0.4500893	best: 0.4500865 (392)	total: 823ms	remaining: 1.26s
394:	learn: 0.4404394	test: 0.4500962	best: 0.4500865 (392)	total: 825ms	remaining: 1.26s
395:	learn: 0.4404206	test: 0.4500980	best: 0.4500865 (392)	total: 828ms	remaining: 1.26s
396:	learn: 0.4404021	test: 0.4500921	best: 0.4500865 (392)	total: 830ms	remaining: 1.26s
397:	learn: 0.4403905	test: 0.4500965	best: 0.4500865 (392)	total: 832ms	remaining: 1.26s
398:	learn: 0.4403771	test: 0.4501035	best: 0.4500865 (392)	total: 833ms	remaining: 1.25s
399:	learn

...

536:	learn: 0.4382288	test: 0.4479772	best: 0.4479603 (526)	total: 2.06s	remaining: 1.77s
537:	learn: 0.4382204	test: 0.4479818	best: 0.4479603 (526)	total: 2.06s	remaining: 1.77s
538:	learn: 0.4381990	test: 0.4479839	best: 0.4479603 (526)	total: 2.06s	remaining: 1.76s
539:	learn: 0.4381839	test: 0.4479864	best: 0.4479603 (526)	total: 2.07s	remaining: 1.76s
540:	learn: 0.4381744	test: 0.4479846	best: 0.4479603 (526)	total: 2.07s	remaining: 1.76s
541:	learn: 0.4381565	test: 0.4479798	best: 0.4479603 (526)	total: 2.07s	remaining: 1.75s
542:	learn: 0.4381412	test: 0.4479790	best: 0.4479603 (526)	total: 2.08s	remaining: 1.75s
543:	learn: 0.4381223	test: 0.4479762	best: 0.4479603 (526)	total: 2.08s	remaining: 1.75s
544:	learn: 0.4381028	test: 0.4479794	best: 0.4479603 (526)	total: 2.09s	remaining: 1.74s
545:	learn: 0.4380850	test: 0.4479858	best: 0.4479603 (526)	total: 2.09s	remaining: 1.74s
546:	learn: 0.4380713	test: 0.4479893	best: 0.4479603 (526)	total: 2.1s	remaining: 1.74s
547:	learn:

645:	learn: 0.4368061	test: 0.4480115	best: 0.4479436 (585)	total: 2.48s	remaining: 1.36s
646:	learn: 0.4367956	test: 0.4480130	best: 0.4479436 (585)	total: 2.48s	remaining: 1.35s
647:	learn: 0.4367881	test: 0.4480136	best: 0.4479436 (585)	total: 2.48s	remaining: 1.35s
648:	learn: 0.4367768	test: 0.4480089	best: 0.4479436 (585)	total: 2.49s	remaining: 1.34s
649:	learn: 0.4367664	test: 0.4480083	best: 0.4479436 (585)	total: 2.49s	remaining: 1.34s
650:	learn: 0.4367589	test: 0.4480091	best: 0.4479436 (585)	total: 2.49s	remaining: 1.34s
651:	learn: 0.4367518	test: 0.4480141	best: 0.4479436 (585)	total: 2.5s	remaining: 1.33s
652:	learn: 0.4367374	test: 0.4480197	best: 0.4479436 (585)	total: 2.5s	remaining: 1.33s
653:	learn: 0.4367237	test: 0.4480212	best: 0.4479436 (585)	total: 2.51s	remaining: 1.32s
654:	learn: 0.4367107	test: 0.4480244	best: 0.4479436 (585)	total: 2.51s	remaining: 1.32s
655:	learn: 0.4366940	test: 0.4480267	best: 0.4479436 (585)	total: 2.51s	remaining: 1.32s
656:	learn: 

749:	learn: 0.4355387	test: 0.4480604	best: 0.4479436 (585)	total: 2.89s	remaining: 963ms
750:	learn: 0.4355282	test: 0.4480622	best: 0.4479436 (585)	total: 2.89s	remaining: 959ms
751:	learn: 0.4355220	test: 0.4480637	best: 0.4479436 (585)	total: 2.9s	remaining: 955ms
752:	learn: 0.4355200	test: 0.4480637	best: 0.4479436 (585)	total: 2.9s	remaining: 951ms
753:	learn: 0.4355092	test: 0.4480634	best: 0.4479436 (585)	total: 2.9s	remaining: 948ms
754:	learn: 0.4354987	test: 0.4480648	best: 0.4479436 (585)	total: 2.91s	remaining: 944ms
755:	learn: 0.4354883	test: 0.4480653	best: 0.4479436 (585)	total: 2.91s	remaining: 941ms
756:	learn: 0.4354696	test: 0.4480763	best: 0.4479436 (585)	total: 2.92s	remaining: 937ms
757:	learn: 0.4354529	test: 0.4480811	best: 0.4479436 (585)	total: 2.92s	remaining: 934ms
758:	learn: 0.4354404	test: 0.4480717	best: 0.4479436 (585)	total: 2.93s	remaining: 930ms
759:	learn: 0.4354249	test: 0.4480752	best: 0.4479436 (585)	total: 2.93s	remaining: 927ms
760:	learn: 0

848:	learn: 0.4343728	test: 0.4481264	best: 0.4479436 (585)	total: 3.28s	remaining: 584ms
849:	learn: 0.4343624	test: 0.4481314	best: 0.4479436 (585)	total: 3.29s	remaining: 580ms
850:	learn: 0.4343493	test: 0.4481310	best: 0.4479436 (585)	total: 3.29s	remaining: 576ms
851:	learn: 0.4343398	test: 0.4481328	best: 0.4479436 (585)	total: 3.29s	remaining: 572ms
852:	learn: 0.4343287	test: 0.4481316	best: 0.4479436 (585)	total: 3.3s	remaining: 568ms
853:	learn: 0.4343159	test: 0.4481348	best: 0.4479436 (585)	total: 3.3s	remaining: 565ms
854:	learn: 0.4343021	test: 0.4481403	best: 0.4479436 (585)	total: 3.31s	remaining: 561ms
855:	learn: 0.4342902	test: 0.4481357	best: 0.4479436 (585)	total: 3.31s	remaining: 558ms
856:	learn: 0.4342760	test: 0.4481341	best: 0.4479436 (585)	total: 3.32s	remaining: 554ms
857:	learn: 0.4342632	test: 0.4481367	best: 0.4479436 (585)	total: 3.32s	remaining: 550ms
858:	learn: 0.4342489	test: 0.4481420	best: 0.4479436 (585)	total: 3.33s	remaining: 547ms
859:	learn: 

955:	learn: 0.4331531	test: 0.4482123	best: 0.4479436 (585)	total: 3.69s	remaining: 170ms
956:	learn: 0.4331449	test: 0.4482072	best: 0.4479436 (585)	total: 3.7s	remaining: 166ms
957:	learn: 0.4331366	test: 0.4482105	best: 0.4479436 (585)	total: 3.7s	remaining: 162ms
958:	learn: 0.4331290	test: 0.4482111	best: 0.4479436 (585)	total: 3.71s	remaining: 158ms
959:	learn: 0.4331158	test: 0.4482151	best: 0.4479436 (585)	total: 3.72s	remaining: 155ms
960:	learn: 0.4331031	test: 0.4482094	best: 0.4479436 (585)	total: 3.72s	remaining: 151ms
961:	learn: 0.4330899	test: 0.4482170	best: 0.4479436 (585)	total: 3.73s	remaining: 147ms
962:	learn: 0.4330745	test: 0.4482174	best: 0.4479436 (585)	total: 3.73s	remaining: 144ms
963:	learn: 0.4330598	test: 0.4482218	best: 0.4479436 (585)	total: 3.74s	remaining: 140ms
964:	learn: 0.4330520	test: 0.4482255	best: 0.4479436 (585)	total: 3.74s	remaining: 136ms
965:	learn: 0.4330376	test: 0.4482267	best: 0.4479436 (585)	total: 3.75s	remaining: 132ms
966:	learn: 

In [110]:
grid_search_result['params']

{'depth': 4, 'l2_leaf_reg': 9, 'learning_rate': 0.03}

In [111]:
cbr = CatBoostRegressor().fit(x_train,y_train) # **grid_search_result['params']
preds = cbr.predict(test_standard)
np.expm1(preds)

Learning rate set to 0.070684
0:	learn: 0.4525383	total: 5.58ms	remaining: 5.58s
1:	learn: 0.4518226	total: 10.6ms	remaining: 5.3s
2:	learn: 0.4511124	total: 15.5ms	remaining: 5.17s
3:	learn: 0.4505467	total: 20.8ms	remaining: 5.17s
4:	learn: 0.4500293	total: 25.5ms	remaining: 5.07s
5:	learn: 0.4495846	total: 30.5ms	remaining: 5.05s
6:	learn: 0.4491648	total: 35.5ms	remaining: 5.04s
7:	learn: 0.4487998	total: 40.6ms	remaining: 5.04s
8:	learn: 0.4484783	total: 46.4ms	remaining: 5.11s
9:	learn: 0.4481320	total: 51.3ms	remaining: 5.08s
10:	learn: 0.4479040	total: 56.2ms	remaining: 5.06s
11:	learn: 0.4476600	total: 61.3ms	remaining: 5.05s
12:	learn: 0.4474370	total: 66ms	remaining: 5.01s
13:	learn: 0.4471510	total: 70.7ms	remaining: 4.98s
14:	learn: 0.4469535	total: 75.9ms	remaining: 4.99s
15:	learn: 0.4467468	total: 80.5ms	remaining: 4.95s
16:	learn: 0.4465332	total: 85.5ms	remaining: 4.94s
17:	learn: 0.4463661	total: 90.8ms	remaining: 4.95s
18:	learn: 0.4461909	total: 95.7ms	remaining: 4

181:	learn: 0.4355502	total: 1.03s	remaining: 4.63s
182:	learn: 0.4354990	total: 1.04s	remaining: 4.63s
183:	learn: 0.4354516	total: 1.04s	remaining: 4.63s
184:	learn: 0.4353645	total: 1.05s	remaining: 4.63s
185:	learn: 0.4353129	total: 1.06s	remaining: 4.63s
186:	learn: 0.4352331	total: 1.06s	remaining: 4.63s
187:	learn: 0.4351801	total: 1.07s	remaining: 4.63s
188:	learn: 0.4351216	total: 1.08s	remaining: 4.64s
189:	learn: 0.4350734	total: 1.09s	remaining: 4.64s
190:	learn: 0.4350329	total: 1.09s	remaining: 4.64s
191:	learn: 0.4349622	total: 1.1s	remaining: 4.64s
192:	learn: 0.4349048	total: 1.11s	remaining: 4.64s
193:	learn: 0.4348211	total: 1.12s	remaining: 4.64s
194:	learn: 0.4347438	total: 1.12s	remaining: 4.64s
195:	learn: 0.4346963	total: 1.13s	remaining: 4.64s
196:	learn: 0.4346408	total: 1.14s	remaining: 4.64s
197:	learn: 0.4345592	total: 1.15s	remaining: 4.64s
198:	learn: 0.4345058	total: 1.15s	remaining: 4.64s
199:	learn: 0.4344654	total: 1.16s	remaining: 4.63s
200:	learn: 0

362:	learn: 0.4255087	total: 2.25s	remaining: 3.94s
363:	learn: 0.4254626	total: 2.26s	remaining: 3.94s
364:	learn: 0.4254232	total: 2.26s	remaining: 3.94s
365:	learn: 0.4253620	total: 2.27s	remaining: 3.93s
366:	learn: 0.4253203	total: 2.28s	remaining: 3.93s
367:	learn: 0.4252622	total: 2.28s	remaining: 3.92s
368:	learn: 0.4252241	total: 2.29s	remaining: 3.91s
369:	learn: 0.4251631	total: 2.29s	remaining: 3.91s
370:	learn: 0.4251246	total: 2.3s	remaining: 3.9s
371:	learn: 0.4250657	total: 2.3s	remaining: 3.89s
372:	learn: 0.4250252	total: 2.31s	remaining: 3.88s
373:	learn: 0.4249872	total: 2.31s	remaining: 3.88s
374:	learn: 0.4249374	total: 2.32s	remaining: 3.87s
375:	learn: 0.4248700	total: 2.33s	remaining: 3.86s
376:	learn: 0.4248340	total: 2.33s	remaining: 3.85s
377:	learn: 0.4247987	total: 2.34s	remaining: 3.85s
378:	learn: 0.4247478	total: 2.34s	remaining: 3.84s
379:	learn: 0.4247204	total: 2.35s	remaining: 3.83s
380:	learn: 0.4246549	total: 2.35s	remaining: 3.82s
381:	learn: 0.4

547:	learn: 0.4173868	total: 3.27s	remaining: 2.7s
548:	learn: 0.4173339	total: 3.28s	remaining: 2.69s
549:	learn: 0.4172879	total: 3.29s	remaining: 2.69s
550:	learn: 0.4172412	total: 3.29s	remaining: 2.68s
551:	learn: 0.4171888	total: 3.3s	remaining: 2.68s
552:	learn: 0.4171380	total: 3.31s	remaining: 2.67s
553:	learn: 0.4170810	total: 3.31s	remaining: 2.67s
554:	learn: 0.4170486	total: 3.32s	remaining: 2.66s
555:	learn: 0.4170266	total: 3.32s	remaining: 2.65s
556:	learn: 0.4169708	total: 3.33s	remaining: 2.65s
557:	learn: 0.4169325	total: 3.33s	remaining: 2.64s
558:	learn: 0.4168971	total: 3.34s	remaining: 2.63s
559:	learn: 0.4168587	total: 3.34s	remaining: 2.63s
560:	learn: 0.4168212	total: 3.35s	remaining: 2.62s
561:	learn: 0.4167591	total: 3.35s	remaining: 2.61s
562:	learn: 0.4167236	total: 3.36s	remaining: 2.61s
563:	learn: 0.4166925	total: 3.37s	remaining: 2.6s
564:	learn: 0.4166733	total: 3.37s	remaining: 2.6s
565:	learn: 0.4166263	total: 3.38s	remaining: 2.59s
566:	learn: 0.41

719:	learn: 0.4109396	total: 4.28s	remaining: 1.66s
720:	learn: 0.4109185	total: 4.28s	remaining: 1.66s
721:	learn: 0.4108799	total: 4.29s	remaining: 1.65s
722:	learn: 0.4108345	total: 4.3s	remaining: 1.65s
723:	learn: 0.4108026	total: 4.3s	remaining: 1.64s
724:	learn: 0.4107773	total: 4.31s	remaining: 1.63s
725:	learn: 0.4107277	total: 4.32s	remaining: 1.63s
726:	learn: 0.4106927	total: 4.32s	remaining: 1.62s
727:	learn: 0.4106646	total: 4.33s	remaining: 1.62s
728:	learn: 0.4106421	total: 4.33s	remaining: 1.61s
729:	learn: 0.4106066	total: 4.34s	remaining: 1.6s
730:	learn: 0.4105791	total: 4.35s	remaining: 1.6s
731:	learn: 0.4105520	total: 4.35s	remaining: 1.59s
732:	learn: 0.4105249	total: 4.36s	remaining: 1.59s
733:	learn: 0.4105032	total: 4.37s	remaining: 1.58s
734:	learn: 0.4104371	total: 4.37s	remaining: 1.58s
735:	learn: 0.4103967	total: 4.38s	remaining: 1.57s
736:	learn: 0.4103866	total: 4.38s	remaining: 1.56s
737:	learn: 0.4103431	total: 4.39s	remaining: 1.56s
738:	learn: 0.41

888:	learn: 0.4052950	total: 5.28s	remaining: 659ms
889:	learn: 0.4052468	total: 5.29s	remaining: 654ms
890:	learn: 0.4052273	total: 5.29s	remaining: 648ms
891:	learn: 0.4051902	total: 5.3s	remaining: 642ms
892:	learn: 0.4051596	total: 5.3s	remaining: 636ms
893:	learn: 0.4051384	total: 5.31s	remaining: 630ms
894:	learn: 0.4050985	total: 5.32s	remaining: 624ms
895:	learn: 0.4050730	total: 5.32s	remaining: 618ms
896:	learn: 0.4050552	total: 5.33s	remaining: 612ms
897:	learn: 0.4050126	total: 5.33s	remaining: 606ms
898:	learn: 0.4049851	total: 5.34s	remaining: 600ms
899:	learn: 0.4049851	total: 5.34s	remaining: 593ms
900:	learn: 0.4049721	total: 5.35s	remaining: 587ms
901:	learn: 0.4049409	total: 5.35s	remaining: 581ms
902:	learn: 0.4049108	total: 5.36s	remaining: 575ms
903:	learn: 0.4048873	total: 5.36s	remaining: 570ms
904:	learn: 0.4048593	total: 5.37s	remaining: 564ms
905:	learn: 0.4048312	total: 5.37s	remaining: 558ms
906:	learn: 0.4048016	total: 5.38s	remaining: 552ms
907:	learn: 0.

array([4.15490949, 3.19377166, 5.72865778, ..., 5.16830712, 4.23585668,
       6.01923061])

In [27]:
cat = CatBoostRegressor()

parameters= {'learning_rate': [0.03,0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9],
            'iterations':[1000,1200],
            'loss_function':['RMSE']}

catcv = GridSearchCV(cat, param_grid = parameters, scoring = rmsle_scorer,cv = 3, n_jobs=-1).fit(x_train,y_train)

0:	learn: 3.1839694	total: 25.1ms	remaining: 30.1s
1:	learn: 3.1727796	total: 48.6ms	remaining: 29.1s
2:	learn: 3.1592252	total: 71.2ms	remaining: 28.4s
3:	learn: 3.1488658	total: 92.9ms	remaining: 27.8s
4:	learn: 3.1394708	total: 118ms	remaining: 28.1s
5:	learn: 3.1330879	total: 143ms	remaining: 28.4s
6:	learn: 3.1247459	total: 166ms	remaining: 28.3s
7:	learn: 3.1170813	total: 189ms	remaining: 28.2s
8:	learn: 3.1106174	total: 214ms	remaining: 28.3s
9:	learn: 3.1046343	total: 237ms	remaining: 28.2s
10:	learn: 3.1002616	total: 260ms	remaining: 28.1s
11:	learn: 3.0922514	total: 283ms	remaining: 28s
12:	learn: 3.0877788	total: 306ms	remaining: 27.9s
13:	learn: 3.0835008	total: 328ms	remaining: 27.8s
14:	learn: 3.0789514	total: 350ms	remaining: 27.7s
15:	learn: 3.0721616	total: 373ms	remaining: 27.6s
16:	learn: 3.0693932	total: 395ms	remaining: 27.5s
17:	learn: 3.0637172	total: 418ms	remaining: 27.5s
18:	learn: 3.0594104	total: 440ms	remaining: 27.3s
19:	learn: 3.0548670	total: 463ms	remai

162:	learn: 2.5836114	total: 4.37s	remaining: 27.8s
163:	learn: 2.5821033	total: 4.4s	remaining: 27.8s
164:	learn: 2.5811503	total: 4.43s	remaining: 27.8s
165:	learn: 2.5772953	total: 4.46s	remaining: 27.8s
166:	learn: 2.5754575	total: 4.49s	remaining: 27.8s
167:	learn: 2.5722519	total: 4.53s	remaining: 27.8s
168:	learn: 2.5674580	total: 4.56s	remaining: 27.8s
169:	learn: 2.5653893	total: 4.59s	remaining: 27.8s
170:	learn: 2.5631771	total: 4.62s	remaining: 27.8s
171:	learn: 2.5613757	total: 4.65s	remaining: 27.8s
172:	learn: 2.5610470	total: 4.68s	remaining: 27.8s
173:	learn: 2.5579756	total: 4.71s	remaining: 27.8s
174:	learn: 2.5561255	total: 4.74s	remaining: 27.8s
175:	learn: 2.5517516	total: 4.78s	remaining: 27.8s
176:	learn: 2.5492549	total: 4.81s	remaining: 27.8s
177:	learn: 2.5460476	total: 4.85s	remaining: 27.8s
178:	learn: 2.5445642	total: 4.88s	remaining: 27.8s
179:	learn: 2.5429788	total: 4.91s	remaining: 27.8s
180:	learn: 2.5407828	total: 4.94s	remaining: 27.8s
181:	learn: 2

322:	learn: 2.2405857	total: 9.3s	remaining: 25.3s
323:	learn: 2.2391135	total: 9.33s	remaining: 25.2s
324:	learn: 2.2367034	total: 9.37s	remaining: 25.2s
325:	learn: 2.2353652	total: 9.4s	remaining: 25.2s
326:	learn: 2.2336270	total: 9.43s	remaining: 25.2s
327:	learn: 2.2300454	total: 9.46s	remaining: 25.1s
328:	learn: 2.2272274	total: 9.49s	remaining: 25.1s
329:	learn: 2.2258079	total: 9.52s	remaining: 25.1s
330:	learn: 2.2242615	total: 9.55s	remaining: 25.1s
331:	learn: 2.2215170	total: 9.59s	remaining: 25.1s
332:	learn: 2.2194500	total: 9.62s	remaining: 25s
333:	learn: 2.2175740	total: 9.65s	remaining: 25s
334:	learn: 2.2166619	total: 9.68s	remaining: 25s
335:	learn: 2.2146978	total: 9.71s	remaining: 25s
336:	learn: 2.2121147	total: 9.74s	remaining: 25s
337:	learn: 2.2088144	total: 9.77s	remaining: 24.9s
338:	learn: 2.2059825	total: 9.8s	remaining: 24.9s
339:	learn: 2.2033768	total: 9.83s	remaining: 24.9s
340:	learn: 2.2024359	total: 9.86s	remaining: 24.8s
341:	learn: 2.2004027	tot

486:	learn: 1.9628297	total: 14.3s	remaining: 20.9s
487:	learn: 1.9611812	total: 14.3s	remaining: 20.9s
488:	learn: 1.9593499	total: 14.4s	remaining: 20.9s
489:	learn: 1.9579950	total: 14.4s	remaining: 20.9s
490:	learn: 1.9569164	total: 14.4s	remaining: 20.8s
491:	learn: 1.9559070	total: 14.5s	remaining: 20.8s
492:	learn: 1.9544084	total: 14.5s	remaining: 20.8s
493:	learn: 1.9509146	total: 14.5s	remaining: 20.7s
494:	learn: 1.9503951	total: 14.5s	remaining: 20.7s
495:	learn: 1.9492666	total: 14.6s	remaining: 20.7s
496:	learn: 1.9478161	total: 14.6s	remaining: 20.7s
497:	learn: 1.9451477	total: 14.6s	remaining: 20.6s
498:	learn: 1.9441371	total: 14.7s	remaining: 20.6s
499:	learn: 1.9432123	total: 14.7s	remaining: 20.6s
500:	learn: 1.9417066	total: 14.7s	remaining: 20.5s
501:	learn: 1.9407113	total: 14.8s	remaining: 20.5s
502:	learn: 1.9384344	total: 14.8s	remaining: 20.5s
503:	learn: 1.9374588	total: 14.8s	remaining: 20.5s
504:	learn: 1.9358212	total: 14.8s	remaining: 20.4s
505:	learn: 

648:	learn: 1.7453875	total: 19.1s	remaining: 16.2s
649:	learn: 1.7448066	total: 19.2s	remaining: 16.2s
650:	learn: 1.7423430	total: 19.2s	remaining: 16.2s
651:	learn: 1.7411650	total: 19.2s	remaining: 16.2s
652:	learn: 1.7397227	total: 19.3s	remaining: 16.1s
653:	learn: 1.7389261	total: 19.3s	remaining: 16.1s
654:	learn: 1.7378629	total: 19.3s	remaining: 16.1s
655:	learn: 1.7364826	total: 19.4s	remaining: 16.1s
656:	learn: 1.7361490	total: 19.4s	remaining: 16s
657:	learn: 1.7335925	total: 19.4s	remaining: 16s
658:	learn: 1.7325829	total: 19.5s	remaining: 16s
659:	learn: 1.7316637	total: 19.5s	remaining: 15.9s
660:	learn: 1.7306193	total: 19.5s	remaining: 15.9s
661:	learn: 1.7301949	total: 19.6s	remaining: 15.9s
662:	learn: 1.7288172	total: 19.6s	remaining: 15.9s
663:	learn: 1.7277379	total: 19.6s	remaining: 15.8s
664:	learn: 1.7268653	total: 19.7s	remaining: 15.8s
665:	learn: 1.7259534	total: 19.7s	remaining: 15.8s
666:	learn: 1.7250410	total: 19.7s	remaining: 15.8s
667:	learn: 1.7237

812:	learn: 1.5609837	total: 24s	remaining: 11.4s
813:	learn: 1.5599345	total: 24s	remaining: 11.4s
814:	learn: 1.5591494	total: 24s	remaining: 11.4s
815:	learn: 1.5580374	total: 24.1s	remaining: 11.3s
816:	learn: 1.5575507	total: 24.1s	remaining: 11.3s
817:	learn: 1.5570320	total: 24.1s	remaining: 11.3s
818:	learn: 1.5560716	total: 24.2s	remaining: 11.2s
819:	learn: 1.5555068	total: 24.2s	remaining: 11.2s
820:	learn: 1.5549003	total: 24.2s	remaining: 11.2s
821:	learn: 1.5533246	total: 24.2s	remaining: 11.1s
822:	learn: 1.5522374	total: 24.3s	remaining: 11.1s
823:	learn: 1.5511230	total: 24.3s	remaining: 11.1s
824:	learn: 1.5501076	total: 24.3s	remaining: 11.1s
825:	learn: 1.5496469	total: 24.4s	remaining: 11s
826:	learn: 1.5488588	total: 24.4s	remaining: 11s
827:	learn: 1.5483864	total: 24.4s	remaining: 11s
828:	learn: 1.5481401	total: 24.5s	remaining: 10.9s
829:	learn: 1.5463399	total: 24.5s	remaining: 10.9s
830:	learn: 1.5457906	total: 24.5s	remaining: 10.9s
831:	learn: 1.5451558	to

971:	learn: 1.4120788	total: 28.6s	remaining: 6.71s
972:	learn: 1.4114758	total: 28.6s	remaining: 6.68s
973:	learn: 1.4108896	total: 28.7s	remaining: 6.65s
974:	learn: 1.4099990	total: 28.7s	remaining: 6.62s
975:	learn: 1.4095381	total: 28.7s	remaining: 6.59s
976:	learn: 1.4089806	total: 28.7s	remaining: 6.56s
977:	learn: 1.4081178	total: 28.8s	remaining: 6.53s
978:	learn: 1.4074048	total: 28.8s	remaining: 6.5s
979:	learn: 1.4067533	total: 28.8s	remaining: 6.46s
980:	learn: 1.4060561	total: 28.8s	remaining: 6.43s
981:	learn: 1.4051863	total: 28.8s	remaining: 6.4s
982:	learn: 1.4043132	total: 28.9s	remaining: 6.37s
983:	learn: 1.4034357	total: 28.9s	remaining: 6.34s
984:	learn: 1.4027161	total: 28.9s	remaining: 6.31s
985:	learn: 1.4020919	total: 28.9s	remaining: 6.28s
986:	learn: 1.4017929	total: 29s	remaining: 6.25s
987:	learn: 1.4009197	total: 29s	remaining: 6.22s
988:	learn: 1.4001069	total: 29s	remaining: 6.19s
989:	learn: 1.3989096	total: 29s	remaining: 6.16s
990:	learn: 1.3977156	

1131:	learn: 1.2855566	total: 32.7s	remaining: 1.97s
1132:	learn: 1.2851316	total: 32.8s	remaining: 1.94s
1133:	learn: 1.2847204	total: 32.8s	remaining: 1.91s
1134:	learn: 1.2839924	total: 32.8s	remaining: 1.88s
1135:	learn: 1.2835745	total: 32.9s	remaining: 1.85s
1136:	learn: 1.2826928	total: 32.9s	remaining: 1.82s
1137:	learn: 1.2819092	total: 32.9s	remaining: 1.79s
1138:	learn: 1.2814924	total: 33s	remaining: 1.76s
1139:	learn: 1.2807922	total: 33s	remaining: 1.74s
1140:	learn: 1.2805250	total: 33s	remaining: 1.71s
1141:	learn: 1.2797499	total: 33s	remaining: 1.68s
1142:	learn: 1.2794521	total: 33.1s	remaining: 1.65s
1143:	learn: 1.2789723	total: 33.1s	remaining: 1.62s
1144:	learn: 1.2779758	total: 33.1s	remaining: 1.59s
1145:	learn: 1.2774911	total: 33.1s	remaining: 1.56s
1146:	learn: 1.2759813	total: 33.2s	remaining: 1.53s
1147:	learn: 1.2755505	total: 33.2s	remaining: 1.5s
1148:	learn: 1.2748247	total: 33.2s	remaining: 1.47s
1149:	learn: 1.2736013	total: 33.2s	remaining: 1.45s
11

In [28]:
catcv.best_params_

{'depth': 10,
 'iterations': 1200,
 'l2_leaf_reg': 3,
 'learning_rate': 0.15,
 'loss_function': 'RMSE'}

In [29]:
cat_pred = catcv.best_estimator_.predict(test_standard)
cat_pred

array([5.62927532, 5.91956295, 8.96372914, ..., 5.6548331 , 5.21305582,
       7.59883146])

In [32]:
for c in range(len(cat_pred)) :
    if cat_pred[c] < 0 :
        cat_pred[c] = 0
    

In [33]:
min(cat_pred)

0.0

In [34]:
submission = pd.read_csv('C:/Users/User/Desktop/대구교통사고예측/sample_submission.csv')
submission['ECLO'] = cat_pred
submission.to_csv('C:/Users/User/Desktop/대구교통사고예측/cat_pred.csv',index=False)
submission

,ID,ECLO
0,ACCIDENT_39609,5.629275
1,ACCIDENT_39610,5.919563
2,ACCIDENT_39611,8.963729
3,ACCIDENT_39612,4.066850
4,ACCIDENT_39613,7.547297
...,...,...
10958,ACCIDENT_50567,7.823839
10959,ACCIDENT_50568,5.602348
10960,ACCIDENT_50569,5.654833
10961,ACCIDENT_50570,5.213056


In [117]:
submission = pd.read_csv('C:/Users/User/Desktop/대구교통사고예측/sample_submission.csv')
submission['ECLO'] = np.expm1(preds)
submission.to_csv('C:/Users/User/Desktop/대구교통사고예측/preds.csv',index=False)
submission

,ID,ECLO
0,ACCIDENT_39609,4.154909
1,ACCIDENT_39610,3.193772
2,ACCIDENT_39611,5.728658
3,ACCIDENT_39612,4.150780
4,ACCIDENT_39613,3.961558
...,...,...
10958,ACCIDENT_50567,5.161630
10959,ACCIDENT_50568,3.976305
10960,ACCIDENT_50569,5.168307
10961,ACCIDENT_50570,4.235857


ECLO = 사망자수 * 10 + 중상자수 * 5 + 경상자수 * 3 + 부상자수 * 1 
# 각자 예측해서 ECLO계산해보기